**7. Crie um software que possa encriptar e decriptar no modo cipher block chaining usando uma das seguintes ci-
fras: módulo affine 256, módulo Hill 256, S-DES, DES. teste os dados para S-deS usando um vetor de inicialização binário de 1010 1010. um texto claro binário de 0000
0001 0010 0011 encriptado com uma chave binária de 01111 11101 deverá dar um texto claro binário de 1111
0100 0000 1011. a decriptação deverá funcionar de modo correspondente.**

*obs.: O exemplo do livro é utilizando o modulo S\-DES e não Affine 256*


In [25]:
def mod_inverse(a, m):
    if gcd(a, m) != 1:
        return None
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m

    while v3 != 0:
        q = u3 // v3
        v1, v2, v3, u1, u2, u3 = (
            u1 - q * v1,
            u2 - q * v2,
            u3 - q * v3,
            v1,
            v2,
            v3,
        )
    return u1 % m

def affine_encrypt(text, a, b):
    encrypted_text = ""
    for char in text:
        char_value = ord(char)
        if 0 <= char_value <= 255:
            encrypted_value = (a * char_value + b) % 256
            encrypted_text += chr(encrypted_value)
        else:
            encrypted_text += char
    return encrypted_text

def affine_decrypt(encrypted_text, a, b):
    decrypted_text = ""
    a_inverse = mod_inverse(a, 256)
    if a_inverse is None:
        return "Chave inválida. 'a' e 256 devem ser primos entre si."

    for char in encrypted_text:
        char_value = ord(char)
        if 0 <= char_value <= 255:
            decrypted_value = (a_inverse * (char_value - b)) % 256
            decrypted_text += chr(decrypted_value)
        else:
            decrypted_text += char
    return decrypted_text

In [26]:
# Exemplo de uso:
mensagem = "Olá, mundo!"
chave_a = 5
chave_b = 12

texto_criptografado = affine_encrypt(mensagem, chave_a, chave_b)
print("Texto criptografado:", texto_criptografado)

texto_descriptografado = affine_decrypt(texto_criptografado, chave_a, chave_b)
print("Texto descriptografado:", texto_descriptografado)

Texto criptografado: (qè¬-U2 7±
Texto descriptografado: Olá, mundo!


In [27]:
from operator import xor

def cbc_encrypt(plaintext, key, iv):
    ciphertext = ""
    block_size = len(iv)
    previous_block = iv

    for i in range(0, len(plaintext), block_size):
        block = plaintext[i:i+block_size]

        # XOR com o bloco IV
        xored_block = ''.join(chr(xor(ord(a),ord(b))) for a, b in zip(block, previous_block))

        # Encripa o bloco resultante utilizando o Affine 256
        encrypted_block = affine_encrypt(xored_block, key[0], key[1])

        ciphertext += encrypted_block
        previous_block = encrypted_block

    return ciphertext

def cbc_decrypt(ciphertext, key, iv):
    plaintext = ""
    block_size = len(iv)
    previous_block = iv

    for i in range(0, len(ciphertext), block_size):
        block = ciphertext[i:i+block_size]

        # Decripta o bloco usando o Affine 256
        decrypted_block = affine_decrypt(block, key[0], key[1])

        # XOR com o bloco IV
        xored_block = ''.join(chr(xor(ord(a),ord(b))) for a, b in zip(decrypted_block, previous_block))

        plaintext += xored_block
        previous_block = block

    return plaintext

In [28]:
# Exemplo de uso:
mensagem = "Olá, mundo!"
chave_a = 5
chave_b = 12
vetor_inicializacao = "abcdefghi"  # IV deve ter o mesmo tamanho do bloco

# Criptografar usando o modo CBC
texto_criptografado = cbc_encrypt(mensagem, (chave_a, chave_b), vetor_inicializacao)
print("Texto criptografado:", texto_criptografado)

# Descriptografar usando o modo CBC
texto_descriptografado = cbc_decrypt(texto_criptografado, (chave_a, chave_b), vetor_inicializacao)
print("Texto descriptografado:", texto_descriptografado)

Texto criptografado: òRteCf*MK
Texto descriptografado: Olá, mundo!
